# Dataset initial exploration

## Introduction

This project uses the **Online Retail II** dataset, which contains all transactions recorded between December 1, 2009, and December 9, 2011, by a UK-based online retail company. The company specializes in selling unique gift items for all occasions, and its main customers include international wholesalers and retailers.

This dataset was obtained from Kaggle and is publicly available at this [link](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci).

## Project objective

The main goal of this analysis is to **explore, clean, and prepare the data** in order to answer key questions about customer purchasing behavior, best-selling products, sales patterns by country, and customer profiles. This will lay the foundation for **visualization and data-driven business decision-making**.

In this first stage, I use Python’s **pandas** library to perform an initial exploration of the dataset, along with data cleaning and basic transformations. Later, these steps will be complemented with SQL queries and interactive Power BI dashboards for a deeper analysis.

## Executive summary

- Final cleaned dataset: **1,054,675 rows** and **15 columns**.
- Removed rows with **null Description** (~0.41%) y **Price = 0**; excluded administrative/commission records.
- **Null Customer ID** handled with `Customer Label = 'unknown'` (only for aggregated metrics; not for customer-level KPIs).
- **InvoiceDate** converted to datetime; derived columns: Year, Month, Day Name, Year_Month.
- Line amount: **LineTotal = Quantity * Price**.
- Dataset ready for visual EDA in the next notebook.

## Key analysis questions

These are some of the questions I will answer by analyzing this dataset:

1. What are the top 10 products by sales?
2. Which countries generate the highest sales?
3. How have sales evolved over time (by month)?
4. Who are the top customers by sales?
5. What percentage of invoices correspond to returns?
6. What is the average ticket per invoice?
7. On which day of the week are sales highest?
8. Are there seasonal or monthly sales patterns?
9. Which products are most frequently returned?
10. Which products are often purchased together?

The purpose of this project is to gain a deeper understanding of sales behavior and customer profiles.

## Descriptive exploration and dataset cleaning

---
### 1. First, loading the data and displaying the first rows to understand what information we have.

In [1]:
# Importing pathlib to handle file paths
from pathlib import Path

# Importing pandas for data analysis
import pandas as pd

# Root folder and relative paths
BASE_DIR = Path.cwd().parent
csv_path = BASE_DIR / 'data' / 'online_retail_II.csv'

# Loading the dataset
df = pd.read_csv(csv_path)

# Displaying the first 5 rows of the DataFrame
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Describing the dataset columns

- `Invoice`: Invoice number, identifies each transaction (if it starts with 'C', it refers to a return).
- `StockCode`: Product code.
- `Description`: Product description.
- `Quantity`: Quantity of products sold (can be negative in the case of a return).
- `InvoiceDate`: Date and time of the invoice.
- `Price`: Sales price (in British pounds).
- `Customer ID`: Unique customer identifier.
- `Country`: Country where the purchase was made.

---
### 2. Performing a deeper exploration to check the number of rows and whether we have null values.

In [2]:
# Displaying DataFrame information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


We can observe that:

- The dataset consists of **1,067,371 rows** and **8 columns**.
- There are 5 columns of type object (text), 1 integer column, and 2 float columns.
- The `InvoiceDate` column is of type **string**, it should be converted to **datetime** if we want to perform time-based analysis.
- The `Description` and `Customer ID` columns contain **null values**.

### 3. Cleaning and standardizing the values of the text columns.

In [3]:
# Removing leading and trailing spaces in all object-type columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()
    
# Converting Invoice, StockCode, and Description columns to uppercase
upper_cols = ['Invoice', 'StockCode', 'Description']

for col in upper_cols:
    df[col] = df[col].str.upper()

df['Description'] = df['Description'].str.lstrip('*')

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


---
### 4. Visualizing the exact number of null values for each column in the dataset.

In [4]:
# Counting null values per column
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

To understand which columns may cause problems for the analysis, we also look at the percentage of null values relative to the total number of rows.

In [5]:
# Calculating the percentage of null values per column
df.isnull().sum() / len(df) * 100

Invoice         0.000000
StockCode       0.000000
Description     0.410541
Quantity        0.000000
InvoiceDate     0.000000
Price           0.000000
Customer ID    22.766873
Country         0.000000
dtype: float64

- The `Description` column has **0.41%** of **null values**, so dropping those rows would be acceptable.
- However, the `Customer ID` column has **22.77%** of **null values**, likely corresponding to unregistered sales (guest customers).
- In this case, we need to find another way to handle the values in the `Customer ID` column.
- Simply deleting those rows would risk **losing important information** for the analysis.

---
### 5. Handling null values

We drop the rows where the `Description` column is null, since a transaction without a product description does not provide useful information for the analysis.

In [6]:
# Count rows before and after dropping nulls in 'Description'
rows_before = len(df)
df = df.dropna(subset=['Description'])
rows_after = len(df)

Checking the dataset after the operation

In [7]:
print(f'The dataset had {rows_before} rows before cleaning')
print(f'The dataset has {rows_after} rows after cleaning')

The dataset had 1067371 rows before cleaning
The dataset has 1062989 rows after cleaning


After removing null values from the `Description`,column, the dataset went from **1,067,371 rows** to **1,062,989 rows**. This process eliminated only **0.41%** of the total rows (4,382 rows), which does not affect the analysis in any meaningful way.

In [8]:
# Checking null values after cleaning
df.isnull().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    238625
Country             0
dtype: int64

In this dataset, the `Customer ID` column has many null values, since not all buyers are registered.
If we replace the null values with **'unknown'**, the column will change from numeric (float) to text (object).
This could limit certain numerical or integrity checks on the original IDs.

To follow best practices and avoid altering the original column, while still allowing numerical analysis in the future, we will create a new column called `Customer Label`:
- This column will hold the `Customer ID` for registered customers and 'unknown' for unregistered or anonymous transactions.
- This way, we can use `Customer Label` for all analyses and groupings by customer, without losing the original information or changing its data type.

In [9]:
# Creating 'Customer Label' column and fill nulls with 'unknown'
df['Customer Label'] = df['Customer ID'].fillna('unknown')

By creating the helper column `Customer Label` we can explicitly flag the **absence of a customer code** with the value **'unknown'** and group those cases in the analysis.

In [10]:
# Checking 'Customer Label' for rows where 'Customer ID' is null
df[df['Customer ID'].isnull()][['Invoice', 'Customer ID', 'Customer Label']].head()

,Invoice,Customer ID,Customer Label
263,489464,NaN,unknown
283,489463,NaN,unknown
284,489467,NaN,unknown
577,489525,NaN,unknown
578,489525,NaN,unknown


Checking the dataset again to verify the absence of null values:

In [11]:
# Verifying absence of null values after creating 'Customer Label'
df.isnull().sum()

Invoice                0
StockCode              0
Description            0
Quantity               0
InvoiceDate            0
Price                  0
Customer ID       238625
Country                0
Customer Label         0
dtype: int64

After this transformation, we can see that the new `Customer Label` column created from `Customer ID` no longer contains null values.
Meanwhile, the original `Customer ID` column remains unchanged, preserving the option for advanced numeric analysis if required.

---
### 6. Dropping rows with price equal to zero

I am removing the records with **price equal to zero**, since they **do not provide relevant information** for business analysis and help keep the dataset more **clean and focused**.

First, I create and display a small DataFrame with the rows that will be removed, to verify that the process is working correctly.

In [12]:
# Creating DataFrame with rows where 'Price' = 0 and show 20 random examples
price_zero = df['Price'] == 0
df_price_zero = df[price_zero]
display(df_price_zero.sample(20, random_state=18))

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label
922483,571112,22637,DAMAGES,-65,2011-10-13 17:36:00,0.0,NaN,United Kingdom,unknown
1035578,579429,20711,LOST IN SPACE,-98,2011-11-29 13:04:00,0.0,NaN,United Kingdom,unknown
269559,515464,22126,GIVEN AWAY,-5000,2010-07-12 15:48:00,0.0,NaN,United Kingdom,unknown
248851,513450,20726,LUNCH BAG WOODLAND,1,2010-06-24 15:03:00,0.0,NaN,United Kingdom,unknown
580650,540978,84050,COUNTED,-310,2011-01-12 15:04:00,0.0,NaN,United Kingdom,unknown
692101,550948,17109D,ADJUSTMENT,14,2011-04-21 15:56:00,0.0,NaN,United Kingdom,unknown
320939,520681,22465,HANGING METAL STAR LANTERN,1,2010-08-27 15:28:00,0.0,NaN,United Kingdom,unknown
1005638,577263,47503H,FOUND,66,2011-11-18 12:31:00,0.0,NaN,United Kingdom,unknown
100429,498888,15056N,WEDDING CO RETURNS?,-700,2010-02-23 13:08:00,0.0,NaN,United Kingdom,unknown
248839,513450,22202,MILK PAN PINK RETROSPOT,1,2010-06-24 15:03:00,0.0,NaN,United Kingdom,unknown


Once verified that the rows to be removed are correct, I proceed with **dropping these rows with Price = 0** from the main dataset.

In [13]:
# Dropping rows where 'Price' equals 0
df = df[df['Price'] != 0]

---
### 7. Converting the `InvoiceDate` column from text to datetime

It is important to perform this type conversion in order to carry out **time-based analyses** (sales by month, year, day of the week, etc.).

In [14]:
# Convert 'InvoiceDate' to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

Next, I review the data types of all columns again to verify that the conversion was successful and that there are no rows with null values.

In [15]:
# Checking data types after conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1061169 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Invoice         1061169 non-null  object        
 1   StockCode       1061169 non-null  object        
 2   Description     1061169 non-null  object        
 3   Quantity        1061169 non-null  int64         
 4   InvoiceDate     1061169 non-null  datetime64[ns]
 5   Price           1061169 non-null  float64       
 6   Customer ID     824293 non-null   float64       
 7   Country         1061169 non-null  object        
 8   Customer Label  1061169 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 81.0+ MB


As shown above, the `InvoiceDate` column is now of type datetime (datetime64[ns]) and contains no null values.

In addition, I create new columns for **day of the week**, **month**, **year** y **and year/month** to facilitate time-series analysis of sales:

In [16]:
# Creating new columns for year, month, day of the week, year-month, and hour from 'InvoiceDate'
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day Name'] = df['InvoiceDate'].dt.day_name()
df['Year_Month'] = df['InvoiceDate'].dt.to_period('M').dt.to_timestamp()
df['Hour'] = df['InvoiceDate'].dt.strftime('%H:00')

This way, we will be able to analyze sales by year, track their chronological evolution, detect possible seasonality, and even identify which day of the week tend to generate the highest sales.

---
### 8. Creating a column with the total amount per line

We add a new column `LineTotal` that represents the total value of each invoice line, calculated as the product of quantity and unit price.  
This column is essential for any revenue analysis, as well as for sales by product, country, customer, or time period.

In [17]:
df['LineTotal'] = df['Quantity'] * df['Price']

Next, I verify the creation of the column and check that the calculations are correct:

In [18]:
# Displaying the first 5 rows
df[['Quantity', 'Price', 'LineTotal']].head()

,Quantity,Price,LineTotal
0,12,6.95,83.4
1,12,6.75,81.0
2,12,6.75,81.0
3,48,2.10,100.8
4,24,1.25,30.0


With the creation of the `LineTotal` column, our dataset is now ready for sales analysis.

---
### 9. Cleaning and standardizing values in the `Country` column

> **Update note 1:**  
> During the exploratory analysis (next notebook), I detected that the value `'EIRE'` should be unified as `'Ireland'`. Therefore, I include this correction in the cleaning process to ensure data consistency.

In [19]:
# Replacing 'EIRE' with 'Ireland'
df['Country'] = df['Country'].replace({'EIRE': 'Ireland'})

Now, I review the **unique values** in the `Country` column to check if there are any other values that need to be transformed:

In [20]:
# Printing unique values in 'Country'
print(df['Country'].unique())

['United Kingdom' 'France' 'USA' 'Belgium' 'Australia' 'Ireland' 'Germany'
 'Portugal' 'Japan' 'Denmark' 'Nigeria' 'Netherlands' 'Poland' 'Spain'
 'Channel Islands' 'Italy' 'Cyprus' 'Greece' 'Norway' 'Austria' 'Sweden'
 'United Arab Emirates' 'Finland' 'Switzerland' 'Unspecified' 'Malta'
 'Bahrain' 'RSA' 'Bermuda' 'Hong Kong' 'Singapore' 'Thailand' 'Israel'
 'Lithuania' 'West Indies' 'Lebanon' 'Korea' 'Brazil' 'Canada' 'Iceland'
 'Saudi Arabia' 'Czech Republic' 'European Community']


From this exploration of unique values, I identified **Unspecified**, **European Community** and **West Indies**, which reflect a lack of geographic specificity.

It is necessary to check what **percentage of the total** these values represent:

In [21]:
# Printing frequency of each value in 'Country'
print(df['Country'].value_counts())

Country
United Kingdom          975156
Ireland                  17861
Germany                  17615
France                   14329
Netherlands               5135
Spain                     3810
Switzerland               3188
Belgium                   3122
Portugal                  2620
Australia                 1910
Channel Islands           1664
Italy                     1534
Norway                    1454
Sweden                    1364
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Unspecified                756
Greece                     663
Japan                      582
Poland                     535
USA                        535
United Arab Emirates       500
Israel                     371
Hong Kong                  364
Singapore                  346
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
RSA                        168


The combined frequency of **Unspecified** and **European Community** is less than **0.08% of the total**, so we can decide either to replace them with **Other** or simply **remove them from the DataFrame**.

In the case of **West Indies**, even though the representation in the DataFrame is very small, it may be useful to keep this value to preserve the geographic area it refers to, since the individual countries do not appear in the list of values.

Another poorly descriptive value in the `Country` column is **RSA**. Following the same logic as with **EIRE**, I will replace it with **South Africa** to **make the data easier to interpret** during visualization.

In [22]:
# Removing 'Unspecified' and 'European Community' from the DataFrame
df = df[~df['Country'].isin(['Unspecified', 'European Community'])]

# Replacing 'RSA' with 'South Africa'
df['Country'] = df['Country'].replace({'RSA': 'South Africa'})

After applying these transformations, I check the updated values in the `Country` column:

In [23]:
# Checking frequency of values after cleaning
print(df['Country'].value_counts())

Country
United Kingdom          975156
Ireland                  17861
Germany                  17615
France                   14329
Netherlands               5135
Spain                     3810
Switzerland               3188
Belgium                   3122
Portugal                  2620
Australia                 1910
Channel Islands           1664
Italy                     1534
Norway                    1454
Sweden                    1364
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Greece                     663
Japan                      582
USA                        535
Poland                     535
United Arab Emirates       500
Israel                     371
Hong Kong                  364
Singapore                  346
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
South Africa               168
Bahrain                    126


With these corrections, the `Country` column is now cleaned and standardized, which will make subsequent sales analysis by country more consistent and reliable.

---
### 10. Proactive cleaning of the remaining DataFrame columns

After identifying and correcting outliers in the `Country` column, it is important to apply the same rigor to the other columns of the DataFrame. I proceed to explore and analyze the values present in other key variables, with the goal of detecting potential inconsistencies, outliers, or recording errors that could affect the quality of subsequent analysis. This proactive approach ensures that the entire dataset is clean and ready for the next steps of analysis.

I will now review the `Invoice`, `StockCode`, and `Description` columns to check for inconsistencies.

According to the dataset documentation, any invoice whose identifier in the `Invoice` column begins with the letter 'C' corresponds to a return. Therefore, **the only records that should have negative `LineTotal` values are those associated with returns**. This criterion allows us to validate data consistency: if there are orders with negative `LineTotal` values in invoices that are not returns, they may represent recording errors that should be corrected or investigated before continuing with the analysis.

In [24]:
# Checking for rows with negative LineTotal and invoices not starting with 'C'
df_invalid = df[(df['LineTotal'] < 0) & (~df['Invoice'].astype(str).str.startswith('C'))]
print(f'Number of invalid negative rows: {len(df_invalid)}')
display(df_invalid)

Number of invalid negative rows: 5


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal
179403,A506401,B,ADJUST BAD DEBT,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom,unknown,2010,4,Thursday,2010-04-01,13:00,-53594.36
276274,A516228,B,ADJUST BAD DEBT,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom,unknown,2010,7,Monday,2010-07-01,11:00,-44031.79
403472,A528059,B,ADJUST BAD DEBT,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom,unknown,2010,10,Wednesday,2010-10-01,12:00,-38925.87
825444,A563186,B,ADJUST BAD DEBT,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,unknown,2011,8,Friday,2011-08-01,14:00,-11062.06
825445,A563187,B,ADJUST BAD DEBT,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,unknown,2011,8,Friday,2011-08-01,14:00,-11062.06


When exploring negative values in `LineTotal` that are not structured as returns, we observe that they correspond to descriptions such as **Adjust bad debt**. These cases represent internal accounting adjustments and **will not be considered** in commercial sales or return analyses, since they are values linked to invoices that are not traditional returns.

In many sales systems it is common to find records that **do not correspond to actual product sales**, but rather to accounting adjustments, internal tests, free samples, or other administrative movements. These records are usually identified in the `Description` column by terms such as **'bad debt', 'sample', 'post', 'manual', 'test', 'check', 'discount' or 'promotion'**.

To ensure that the commercial analysis reflects only real sales and returns, I identify and exclude these records from the cleaned dataset.

In [25]:
# List of keywords that indicate administrative or special records
keywords = ['bad debt', 'sample', 'post', 'manual', 'test', 'check', 'discount', 'promotion']

# Boolean filter for rows with special descriptions
special_filter = df['Description'].str.contains('|'.join(keywords), case=False, na=False)

# Creating DataFrame with special records
df_special = df[special_filter]

# Checking the size of the special DataFrame
print(f'Number of special rows in the DataFrame: {len(df_special)}')

# Taking a random sample of 20 rows from the filtered DataFrame
display(df_special.sample(20, random_state=18))

Number of special rows in the DataFrame: 7295


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal
354631,523893,22944,CHRISTMAS METAL POSTCARD WITH BELLS,12,2010-09-24 14:56:00,1.25,13381.0,United Kingdom,13381.0,2010,9,Friday,2010-09-01,14:00,15.00
178340,506285,M,MANUAL,3,2010-04-28 16:49:00,1.45,16767.0,United Kingdom,16767.0,2010,4,Wednesday,2010-04-01,16:00,4.35
574958,540519,POST,POSTAGE,2,2011-01-09 12:12:00,28.00,12793.0,Portugal,12793.0,2011,1,Sunday,2011-01-01,12:00,56.00
685313,C550365,POST,POSTAGE,-1,2011-04-18 11:31:00,11.00,12731.0,France,12731.0,2011,4,Monday,2011-04-01,11:00,-11.00
405775,528179,M,MANUAL,3,2010-10-21 10:49:00,0.85,14227.0,United Kingdom,14227.0,2010,10,Thursday,2010-10-01,10:00,2.55
306054,519238,21407,BROWN CHECK CAT DOORSTOP,6,2010-08-15 13:28:00,4.25,13405.0,United Kingdom,13405.0,2010,8,Sunday,2010-08-01,13:00,25.50
57087,494495,21407,BROWN CHECK CAT DOORSTOP,1,2010-01-14 17:43:00,8.47,NaN,United Kingdom,unknown,2010,1,Thursday,2010-01-01,17:00,8.47
796623,560651,M,MANUAL,1,2011-07-20 11:38:00,451.42,15802.0,United Kingdom,15802.0,2011,7,Wednesday,2011-07-01,11:00,451.42
396355,527392,M,MANUAL,1,2010-10-17 13:28:00,3.75,14044.0,United Kingdom,14044.0,2010,10,Sunday,2010-10-01,13:00,3.75
767908,C558347,S,SAMPLES,-1,2011-06-28 14:47:00,9.90,NaN,United Kingdom,unknown,2011,6,Tuesday,2011-06-01,14:00,-9.90


In the list of identified descriptions, we can see that there are some **false positives**, such as 'BROWN CHECK CAT DOORSTOP' or 'VICTORIAN METAL POSTCARD SPRING', which are in fact **real sales** and not administrative records.

Therefore, it is necessary to review the unique values in this filtered DataFrame to identify which other **legitimate products** are being incorrectly included by the filter, and refine the exclusion logic if needed.

In [26]:
# Counting the frequency of special values
df_special['Description'].value_counts()

Description
POSTAGE                                2112
DOTCOM POSTAGE                         1439
MANUAL                                 1418
BROWN CHECK CAT DOORSTOP                472
VICTORIAN  METAL POSTCARD SPRING        434
SET OF 12  VINTAGE POSTCARD SET         263
DISCOUNT                                177
CHRISTMAS METAL POSTCARD WITH BELLS     172
POSTE FRANCE CUSHION COVER              155
SUNSET CHECK HAMMOCK                    115
SAMPLES                                 104
PAIR PADDED HANGERS PINK CHECK          100
WRAP ALPHABET POSTER                     64
VINTAGE POST OFFICE CABINET              54
VICTORIAN METAL POSTCARD CHRISTMAS       49
YELLOW EASTER EGG HUNT START POST        49
GREEN EASTER EGG HUNT START POST         29
BLUE EASTER EGG HUNT START POST          27
MULTICOLOUR CRUSOE CHECK LAMPSHADE       17
THIS IS A TEST PRODUCT.                  14
BLUE CHECK BAG W HANDLE 34X20CM          11
BLUE CRUSOE CHECK LAMPSHADE               7
SET 10 CARDS PERFECT

It can be confirmed that the keywords **"Check"** and **"Post"** were mistakenly included by the current filter, since the results for **"Check"** correspond to actual products. From this analysis, it is clear that the description that should be filtered is actually **"Postage"**. I will use these findings to refine the cleaning filter.

In [27]:
# Updating the list of keywords indicating administrative or special records
keywords = ['bad debt', 'sample', 'postage', 'manual', 'test', 'discount', 'promotion']

# Refining the filter for rows with special descriptions
revised_filter = df['Description'].str.contains('|'.join(keywords), case=False, na=False)

# Updating the DataFrame with special records
df_special = df[revised_filter]

# Displaying the frequency of unique values in the revised special DataFrame
display(df_special['Description'].value_counts())

Description
POSTAGE                    2112
DOTCOM POSTAGE             1439
MANUAL                     1418
DISCOUNT                    177
SAMPLES                     104
THIS IS A TEST PRODUCT.      14
ADJUST BAD DEBT               6
Name: count, dtype: int64

After refining the filter and reviewing the results, I confirm that **no incorrect values are being included** by mistake. I can therefore proceed with the **safe removal** of these records from the dataset.

In [28]:
# Removing special records from the DataFrame
df = df[~revised_filter]

These administrative or special records identified by the refined filter are now removed from the dataset, ensuring that subsequent analysis is based exclusively on **real sales and returns**.

Next, I review the `Description` column for potential inconsistencies.

In [29]:
# Checking values in the 'Description' column
print("Number of unique products:", df['Description'].nunique())
print("\nTop 20 most frequent products:\n", df['Description'].value_counts().head(20))
print("\nRandom sample of product descriptions:\n", df['Description'].sample(20, random_state=18))

Number of unique products: 5365

Top 20 most frequent products:
 Description
WHITE HANGING HEART T-LIGHT HOLDER    5912
REGENCY CAKESTAND 3 TIER              4404
JUMBO BAG RED RETROSPOT               3465
ASSORTED COLOUR BIRD ORNAMENT         2954
PARTY BUNTING                         2763
STRAWBERRY CERAMIC TRINKET BOX        2608
LUNCH BAG  BLACK SKULL.               2528
JUMBO STORAGE BAG SUKI                2431
HEART OF WICKER SMALL                 2313
JUMBO SHOPPER VINTAGE RED PAISLEY     2295
60 TEATIME FAIRY CAKE CASES           2266
PAPER CHAIN KIT 50'S CHRISTMAS        2215
LUNCH BAG SPACEBOY DESIGN             2204
REX CASH+CARRY JUMBO SHOPPER          2196
HOME BUILDING BLOCK WORD              2192
WOODEN FRAME ANTIQUE WHITE            2189
LUNCH BAG CARS BLUE                   2183
NATURAL SLATE HEART CHALKBOARD        2150
BAKING SET 9 PIECE RETROSPOT          2146
WOODEN PICTURE FRAME WHITE FINISH     2114
Name: count, dtype: int64

Random sample of product description

And I do the same with the `StockCode` column:

In [30]:
# Checking values in the 'StockCode' column
print("Number of unique product codes:", df['StockCode'].nunique())
print("\nTop 20 most frequent product codes:\n", df['StockCode'].value_counts().head(20))
print("\nRandom sample of product codes:\n", df['StockCode'].sample(20, random_state=18))

Number of unique product codes: 4752

Top 20 most frequent product codes:
 StockCode
85123A    5921
22423     4404
85099B    4235
21212     3315
20725     3255
84879     2954
47566     2763
21232     2742
22383     2539
22197     2538
20727     2528
21931     2431
22386     2347
22469     2313
22411     2295
84991     2266
22382     2249
22384     2226
21080     2223
22086     2215
Name: count, dtype: int64

Random sample of product codes:
 1006158     23296
264977     84970L
836688      22382
871276     84971S
927174     82494L
991023      22138
384228      22271
814995      23049
826389      21080
354560      22582
293768      21928
742701      21933
1015035     23169
222051      21213
993519      20725
761952      23002
490880      22749
1005487     22083
398088      22740
50709       21680
Name: StockCode, dtype: object


When reviewing these columns, no **outliers** are found, so I continue with the dataset cleaning process.

> **Update note 2:**  
> During exploratory analysis (next notebook), I detected values such as **"AMAZON FEE"** and **"Bank Charges"** which **do not represent real sales**, so these will be removed from the dataset. These can be easily identified by their `StockCode` values "AMAZONFEE" and "BANKCHARGES".

> **Update note 3:**  
> I also detected additional adjustment values in the `StockCode` column with the labels **"ADJUST"** and **"ADJUST2"**, which will also be removed from the DataFrame.

> **Update note 4:**  
> During phase 3 of the project (Power BI Dashboard), in the visual validation process, I identified the presence of the `StockCode` value **"CRUK"**, associated with the description **"CRUK commission"**, as well as the value **"23444"** (urgent next-day shipping charge) and **"PADS"**(symbolic value of 0.001). These records **do not represent sales or returns**, but rather administrative, logistical, or internal commission charges. To preserve the integrity of the analysis, these values are removed from the DataFrame and the criteria are documented for future dataset updates.

In [31]:
# Removing rows that do not represent real sales
df = df[~df['StockCode'].isin(['AMAZONFEE', 'BANK CHARGES', 'ADJUST', 'ADJUST2', 'CRUK', '23444', 'PADS'])]

I now verify whether these rows were successfully removed:

In [32]:
# Checking if these rows were succesfully removed
suspects = df[df['StockCode'].isin(['AMAZONFEE', 'BANK CHARGES','ADJUST', 'ADJUST2', 'CRUK', '23444', 'PADS'])]
display(suspects)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal


As expected, the result is an empty DataFrame, which indicates that the removal was successful.

> **Update note 5:**  
> During exploratory analysis, I detected records labeled as **"Gift Voucher"**, which represent the sale of gift cards/vouchers and **do not correspond to direct sales of physical products or services consumed at the time**. After careful consideration, I decided to remove these records from the dataset to avoid artificially inflating sales metrics, since gift vouchers represent a future commitment rather than immediate effective revenue. This is especially relevant in e-commerce sales analysis, where the focus is on measuring actual consumption.

In [33]:
# Counting rows before and after removing 'Gift Voucher' values
print('Rows before cleaning: ', len(df))
print('Rows containing "gift voucher" cleaned: ', df['Description'].str.contains('gift voucher', case=False, na=False).sum())
df = df[~df['Description'].str.contains('gift voucher', case=False, na=False)]

print('Rows after cleaning: ', len(df))

Rows before cleaning:  1054754
Rows containing "gift voucher" cleaned:  79
Rows after cleaning:  1054675


> **Update note 6:**  
>  During phase 3 of the project (Power BI Dashboard), I noticed that some **`StockCode`** values were associated with multiple entries in the **`Description`** column. This is likely due to data entry errors in the dataset. I will normalize these descriptions to the **most frequent value** for each `StockCode`.

In [34]:
# Creating a copy to avoid warnings
df = df.copy()

# Keeping the original 'Description' column for traceability
df['Description_raw'] = df['Description']

# Normalizing values in the 'Description_raw' column
desc_norm = (
    df['Description_raw'].str.upper().str.strip().str.replace(r'\s+', ' ', regex=True)
)

# Calculating the most frequent Description for each StockCode
mode_map = (
    pd.concat([df['StockCode'], desc_norm], axis=1)
      .groupby('StockCode', as_index=True)['Description_raw']
      .agg(lambda s: s.value_counts(dropna=True).idxmax())
)

# Creating a cleaned Description column and replacing the original one
df['Description_clean'] = df['StockCode'].map(mode_map)
df['Description'] = df['Description_clean']

# Checking how many unique values were reduced
before = df['Description_raw'].nunique(dropna=True)
after  = df['Description'].nunique(dropna=True)

print(f"Unique descriptions before:  {before:,}")
print(f"Unique descriptions after: {after:,}")
print(f"Reduction: {before - after:,}")

# Displaying a table with conflicting StockCodes and Descriptions
dup_check = (df.groupby('StockCode')['Description_raw']
               .nunique(dropna=True)
               .rename('n_desc'))
conflict_codes = dup_check[dup_check > 1].index

df_conflicts = (
    df.loc[df['StockCode'].isin(conflict_codes),
           ['StockCode','Description_raw','Description']]
      .drop_duplicates()
      .sort_values(['StockCode','Description_raw'])
)

# Checking the first 30 resolved conflicts
display(df_conflicts.head(30))

print(f'Number of distinct StockCodes: {df['StockCode'].nunique()}')
print(f'Number of distinct Products: {df['Description'].nunique()}')


Unique descriptions before:  5,348
Unique descriptions after: 4,697
Reduction: 651


,StockCode,Description_raw,Description
548898,15058A,BLUE POLKADOT GARDEN PARASOL,BLUE POLKADOT GARDEN PARASOL
67142,15058A,BLUE WHITE SPOTS GARDEN PARASOL,BLUE POLKADOT GARDEN PARASOL
461044,15058B,PINK POLKADOT GARDEN PARASOL,PINK POLKADOT GARDEN PARASOL
54894,15058B,PINK WHITE SPOTS GARDEN PARASOL,PINK POLKADOT GARDEN PARASOL
314800,16012,FOOD/DRINK SPONGE STICKERS,FOOD/DRINK SPONGE STICKERS
4157,16012,FOOD/DRINK SPUNGE STICKERS,FOOD/DRINK SPONGE STICKERS
181581,16151A,FLOWER DES BLUE HANDBAG/ORANG HANDL,FLOWERS HANDBAG BLUE AND ORANGE
710756,16151A,FLOWERS HANDBAG BLUE AND ORANGE,FLOWERS HANDBAG BLUE AND ORANGE
663541,16156L,WRAP CAROUSEL,"WRAP, CAROUSEL"
49282,16156L,"WRAP, CAROUSEL","WRAP, CAROUSEL"


Number of distinct StockCodes: 4737
Number of distinct Products: 4697


---
### 11. Ordering the dataset columns

Although it is not strictly necessary for the columns to be ordered in a logical way when working in pandas, it is considered good practice to organize the dataset columns for later export to Excel, SQL, or for building visualizations in Power BI as we will do in the following steps.

To achieve this, I use the following approach:  
- Place the `LineTotal` column next to `Price`.  
- Place the `Customer Label` column to the right of its reference column `Customer ID`.  
- Place the date-related columns immediately after `InvoiceDate`.

In [35]:
# Reordering dataset columns
df.insert(df.columns.get_loc('Price') + 1, 'LineTotal', df.pop('LineTotal'))
df.insert(df.columns.get_loc('Customer ID') + 1, 'Customer Label', df.pop('Customer Label'))
df.insert(df.columns.get_loc('InvoiceDate') + 1, 'Hour', df.pop('Hour'))
df.insert(df.columns.get_loc('InvoiceDate') + 2, 'Day Name', df.pop('Day Name'))
df.insert(df.columns.get_loc('InvoiceDate') + 3, 'Month', df.pop('Month'))
df.insert(df.columns.get_loc('InvoiceDate') + 4, 'Year', df.pop('Year'))
df.insert(df.columns.get_loc('InvoiceDate') + 5, 'Year_Month', df.pop('Year_Month'))

After completing the cleaning process, I reset the dataset index:

In [36]:
# Reseting dataset index
df.reset_index(drop=True, inplace=True)

Finally, I review how the dataset columns are ordered:

In [37]:
# Displaying the first 5 rows
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Hour,Day Name,Month,Year,Year_Month,Price,LineTotal,Customer ID,Customer Label,Country,Description_raw,Description_clean
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.95,83.4,13085.0,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.75,81.0,13085.0,13085.0,United Kingdom,PINK CHERRY LIGHTS,PINK CHERRY LIGHTS
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.75,81.0,13085.0,13085.0,United Kingdom,WHITE CHERRY LIGHTS,WHITE CHERRY LIGHTS
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,2.10,100.8,13085.0,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE","RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,1.25,30.0,13085.0,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX,STRAWBERRY CERAMIC TRINKET BOX


In [38]:
# Displaying DataFrame information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054675 entries, 0 to 1054674
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   Invoice            1054675 non-null  object        
 1   StockCode          1054675 non-null  object        
 2   Description        1054675 non-null  object        
 3   Quantity           1054675 non-null  int64         
 4   InvoiceDate        1054675 non-null  datetime64[ns]
 5   Hour               1054675 non-null  object        
 6   Day Name           1054675 non-null  object        
 7   Month              1054675 non-null  int32         
 8   Year               1054675 non-null  int32         
 9   Year_Month         1054675 non-null  datetime64[ns]
 10  Price              1054675 non-null  float64       
 11  LineTotal          1054675 non-null  float64       
 12  Customer ID        820166 non-null   float64       
 13  Customer Label     1054675 

Now we have an organized and cleaned dataset, ready for queries and analysis, and prepared for export and sharing with collaborators.

---
### 12. Saving the cleaned dataset in .csv format

In [39]:
# Saving dataset in .csv format
csv_save_path = BASE_DIR / 'data' / 'online_retail_clean.csv'
df.to_csv(csv_save_path, index=False)

Saving the cleaned dataset in a separate file is a best practice in any data analysis project. This allows for:

- **Reproducibility**: Others can continue the project starting from a cleaned dataset without having to repeat the initial cleaning steps.
- **Time efficiency**: If the analysis or visualizations need to be redone, it is much faster to load the cleaned dataset rather than process the raw file each time.
- **Safety and version control**: Keeping the raw data separate from the cleaned data helps prevent accidental errors and makes it easier to compare results before and after the cleaning process.
- **Scalability**: Enables different team members to work on advanced analyses without worrying about inconsistencies caused by non-replicated cleaning steps.

This ensures a professional and efficient workflow throughout the entire project.

---
## Conclusion:

- The dataset was **cleaned of null values** in `Description` by removing those rows.
- Null values in `Customer ID` were handled by creating the **helper column** `Customer Label` and filling nulls with the value **'unknown'**.
- Rows with **Price = 0** were removed as they did not contribute to the overall analysis.
- The `InvoiceDate` column was converted to **datetime type** to facilitate **time-based analysis**.
- From the `InvoiceDate` column, we extracted **four additional columns** (`Year`, `Month`, `Day Name` and `Year_Month`) to provide more precise insights for time analysis.
- The `LineTotal` column was created to calculate the total amount of each invoice line.
- The `Country` column was standardized to improve **readability** and less specific values were removed.
- **Administrative records** and **various commission entries** were filtered out to ensure the dataset reflects **real transactions**.
- Columns were organized in a coherent order for later **export and collaboration**.
- The **final dataset** contains **15 columns and 1,054,675 rows**.